<a href="https://colab.research.google.com/github/loisll/MMAI984/blob/main/lois_trainmodel_Team_Project_VQA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [104]:
!pip install torch torchvision transformers
!pip install tqdm

STEP 1: Import libraries

In [105]:
# step 1: Import libraries
import pandas as pd
from PIL import Image
import json
import matplotlib.pyplot as plt
import seaborn as sns
import os
import numpy as np
import zipfile
from google.colab import drive
import random
import tensorflow as tf

from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import cv2
import re
from tqdm import tqdm

In [106]:
# Define the data path
drive.mount('/content/drive')
data_path = '/content/drive/My Drive/Colab Notebooks/'


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


STEP 2: LOAD TRAIN DATA

In [107]:
# Define function for loading questions and annotations

def load_data(data_file, feature):

  # Check if the file exists
  if os.path.exists(data_file):
    print("File found:", data_file)

    # Load the JSON file using the json module
    with open(data_file, 'r') as f:
        data = json.load(f)

    # Convert the JSON data to a DataFrame
    # questions = pd.DataFrame(data)

    # Flatten the JSON structure
    data = pd.json_normalize(data[feature])

    # Question preprocessing

    print("Data loaded successfully")
  else:
    print("File not found:", data_file)

  return data

*Load and prepare the training dataset*

In [108]:
# Load training questions

# Define the questions file
train_questions_file = os.path.join(data_path, 'train2015/MultipleChoice_abstract_v002_train2015_questions.json')
#train_questions_file = os.path.join(data_path)
train_questions_feature = 'questions'

train_questions = load_data(train_questions_file, train_questions_feature )

# Check the first 5 rows
train_questions

File found: /content/drive/My Drive/Colab Notebooks/train2015/MultipleChoice_abstract_v002_train2015_questions.json
Data loaded successfully


,image_id,question,multiple_choices,question_id
0,11779,Who looks happier?,"[alive, 1, woman, purple, 2, yes, white, boy, ...",117792
1,11779,Where is the woman sitting?,"[3, no, blue, red, 1, slide, monkey bars, jump...",117790
2,11779,Where is the man sitting?,"[away, yes, blue, 1, 2, mouse, couch, no, yell...",117791
3,5536,Is this man hungry?,"[water, yellow, 4, running, blue, pouring, out...",55360
4,5536,What kind of drink is that?,"[wine, girl would fall, soda, white, yes, coke...",55361
...,...,...,...,...
59995,11695,What color is the log the little girl sitting on?,"[tan, brown, white, 1, running, red, 4, pink, ...",116950
59996,11695,Does the boy want to go on the seesaw?,"[red, no table, fetch, kicking soccer ball, ye...",116951
59997,13790,What animal is between the two men?,"[dog, 4, yellow, 1, collectors, no, red, yes, ...",137900
59998,13790,What food is by the picnic basket?,"[4, she is happy, grapes, pizza, white, becaus...",137901


In [109]:
# Load trainning annotations

# Define the annotations file
train_annotations_file = os.path.join(data_path, 'train2015/abstract_v002_train2015_annotations.json')
train_annotations_feature = 'annotations'
train_annotations = load_data(train_annotations_file, train_annotations_feature)

#print(train_annotations_file[annotations])
# Check the first 5 rows
train_annotations

File found: /content/drive/My Drive/Colab Notebooks/train2015/abstract_v002_train2015_annotations.json
Data loaded successfully


,question_type,multiple_choice_answer,answers,image_id,answer_type,question_id
0,who,man,"[{'answer': 'old person', 'answer_confidence':...",11779,other,117792
1,where is the,blanket,"[{'answer': 'on blanket', 'answer_confidence':...",11779,other,117790
2,where is the,bench,"[{'answer': 'on bench', 'answer_confidence': '...",11779,other,117791
3,is this,yes,"[{'answer': 'yes', 'answer_confidence': 'yes',...",5536,yes/no,55360
4,what kind of,soda,"[{'answer': 'water', 'answer_confidence': 'no'...",5536,other,55361
...,...,...,...,...,...,...
59995,what color is the,brown,"[{'answer': 'brown', 'answer_confidence': 'yes...",11695,other,116950
59996,does the,yes,"[{'answer': 'yes', 'answer_confidence': 'yes',...",11695,yes/no,116951
59997,what animal is,cat,"[{'answer': 'cat', 'answer_confidence': 'yes',...",13790,other,137900
59998,what,watermelon,"[{'answer': 'watermelon, sandwich, burger', 'a...",13790,other,137901


In [110]:

# Load trainning captions

# Define the annotations file
train_captions_file = os.path.join(data_path, 'train2015/captions_abstract_v002_train2015.json')
train_captions_feature = 'images'
train_captions = load_data(train_captions_file, train_captions_feature)

#print(train_captions_file[annotations])
# Check the first 5 rows
train_captions

File found: /content/drive/My Drive/Colab Notebooks/train2015/captions_abstract_v002_train2015.json
Data loaded successfully


,url,file_name,image_id,width,height
0,http://visualqa.org/data/abstract_v002/scene_i...,abstract_v002_train2015_000000000000.png,0,700,400
1,http://visualqa.org/data/abstract_v002/scene_i...,abstract_v002_train2015_000000000001.png,1,700,400
2,http://visualqa.org/data/abstract_v002/scene_i...,abstract_v002_train2015_000000000002.png,2,700,400
3,http://visualqa.org/data/abstract_v002/scene_i...,abstract_v002_train2015_000000000003.png,3,700,400
4,http://visualqa.org/data/abstract_v002/scene_i...,abstract_v002_train2015_000000000004.png,4,700,400
...,...,...,...,...,...
19995,http://visualqa.org/data/abstract_v002/scene_i...,abstract_v002_train2015_000000019995.png,19995,700,400
19996,http://visualqa.org/data/abstract_v002/scene_i...,abstract_v002_train2015_000000019996.png,19996,700,400
19997,http://visualqa.org/data/abstract_v002/scene_i...,abstract_v002_train2015_000000019997.png,19997,700,400
19998,http://visualqa.org/data/abstract_v002/scene_i...,abstract_v002_train2015_000000019998.png,19998,700,400


In [111]:
# Load trainning captions

# Define the annotations file
train_OpenEnded_file = os.path.join(data_path, 'train2015/OpenEnded_abstract_v002_train2015_questions.json')
train_OpenEnded_feature = 'questions'
train_OpenEnded = load_data(train_OpenEnded_file, train_OpenEnded_feature)

#print(train_captions_file[annotations])
# Check the first 5 rows
train_OpenEnded

File found: /content/drive/My Drive/Colab Notebooks/train2015/OpenEnded_abstract_v002_train2015_questions.json
Data loaded successfully


,question,image_id,question_id
0,Who looks happier?,11779,117792
1,Where is the woman sitting?,11779,117790
2,Where is the man sitting?,11779,117791
3,Is this man hungry?,5536,55360
4,What kind of drink is that?,5536,55361
...,...,...,...
59995,What color is the log the little girl sitting on?,11695,116950
59996,Does the boy want to go on the seesaw?,11695,116951
59997,What animal is between the two men?,13790,137900
59998,What food is by the picnic basket?,13790,137901


2.2 MERGE TRAIN DATA

In [112]:
# Merge questions and answers

#train_data = pd.merge(train_questions, train_annotations, on='question_id')
train_data = pd.merge(train_questions, train_annotations, on=["image_id", "question_id"])

df_train = train_data.merge(train_captions, on='image_id')
df_train

,image_id,question,multiple_choices,question_id,question_type,multiple_choice_answer,answers,answer_type,url,file_name,width,height
0,11779,Who looks happier?,"[alive, 1, woman, purple, 2, yes, white, boy, ...",117792,who,man,"[{'answer': 'old person', 'answer_confidence':...",other,http://visualqa.org/data/abstract_v002/scene_i...,abstract_v002_train2015_000000011779.png,700,400
1,11779,Where is the woman sitting?,"[3, no, blue, red, 1, slide, monkey bars, jump...",117790,where is the,blanket,"[{'answer': 'on blanket', 'answer_confidence':...",other,http://visualqa.org/data/abstract_v002/scene_i...,abstract_v002_train2015_000000011779.png,700,400
2,11779,Where is the man sitting?,"[away, yes, blue, 1, 2, mouse, couch, no, yell...",117791,where is the,bench,"[{'answer': 'on bench', 'answer_confidence': '...",other,http://visualqa.org/data/abstract_v002/scene_i...,abstract_v002_train2015_000000011779.png,700,400
3,5536,Is this man hungry?,"[water, yellow, 4, running, blue, pouring, out...",55360,is this,yes,"[{'answer': 'yes', 'answer_confidence': 'yes',...",yes/no,http://visualqa.org/data/abstract_v002/scene_i...,abstract_v002_train2015_000000005536.png,700,400
4,5536,What kind of drink is that?,"[wine, girl would fall, soda, white, yes, coke...",55361,what kind of,soda,"[{'answer': 'water', 'answer_confidence': 'no'...",other,http://visualqa.org/data/abstract_v002/scene_i...,abstract_v002_train2015_000000005536.png,700,400
...,...,...,...,...,...,...,...,...,...,...,...,...
59995,11695,What color is the log the little girl sitting on?,"[tan, brown, white, 1, running, red, 4, pink, ...",116950,what color is the,brown,"[{'answer': 'brown', 'answer_confidence': 'yes...",other,http://visualqa.org/data/abstract_v002/scene_i...,abstract_v002_train2015_000000011695.png,700,400
59996,11695,Does the boy want to go on the seesaw?,"[red, no table, fetch, kicking soccer ball, ye...",116951,does the,yes,"[{'answer': 'yes', 'answer_confidence': 'yes',...",yes/no,http://visualqa.org/data/abstract_v002/scene_i...,abstract_v002_train2015_000000011695.png,700,400
59997,13790,What animal is between the two men?,"[dog, 4, yellow, 1, collectors, no, red, yes, ...",137900,what animal is,cat,"[{'answer': 'cat', 'answer_confidence': 'yes',...",other,http://visualqa.org/data/abstract_v002/scene_i...,abstract_v002_train2015_000000013790.png,700,400
59998,13790,What food is by the picnic basket?,"[4, she is happy, grapes, pizza, white, becaus...",137901,what,watermelon,"[{'answer': 'watermelon, sandwich, burger', 'a...",other,http://visualqa.org/data/abstract_v002/scene_i...,abstract_v002_train2015_000000013790.png,700,400


**STEP 3 : EDA**

From the above examples, we can see that most of the questions and answers are simple and clean text but some questions contain punctuation, common word contractions like what’s, it’s, they’re, etc, and noun contractions like guy’s, man’s, dog’s, etc., and some answers also contain punctuation. Hence, we need to perform the data cleaning operation on the question and answer dataset and expand contractions before performing EDA.

In [113]:

def decontractions(phrase):
    """decontracted takes text and convert contractions into natural form.
     ref: https://stackoverflow.com/questions/19790188/expanding-english-language-contractions-in-python/47091490#47091490"""
    # specific
    phrase = re.sub(r"won\'t", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)
    phrase = re.sub(r"won\’t", "will not", phrase)
    phrase = re.sub(r"can\’t", "can not", phrase)

    phrase = re.sub(r"he\'s", "he is", phrase)
    phrase = re.sub(r"she\'s", "she is", phrase)
    phrase = re.sub(r"it\'s", "it is", phrase)

    phrase = re.sub(r"he\’s", "he is", phrase)
    phrase = re.sub(r"she\’s", "she is", phrase)
    phrase = re.sub(r"it\’s", "it is", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)

    phrase = re.sub(r"n\’t", " not", phrase)
    phrase = re.sub(r"\’re", " are", phrase)
    phrase = re.sub(r"\’d", " would", phrase)
    phrase = re.sub(r"\’ll", " will", phrase)
    phrase = re.sub(r"\’t", " not", phrase)
    phrase = re.sub(r"\’ve", " have", phrase)
    phrase = re.sub(r"\’m", " am", phrase)

    return phrase


def text_preprocess(text):
    text = text.lower()
    text = decontractions(text) # replace contractions into natural form
    text = re.sub('[-,:]', ' ', text) # replace the character "-" "," with space
    text = re.sub("(?!<=\d)(\.)(?!\d)", '', text) # remove the character ".", except from floating numbers
    text = re.sub('[^A-Za-z0-9. ]+', '', text) # remove all punctuation, except A-Za-z0-9
    text = re.sub(' +', ' ', text) # remove extra space
    return text

# Question and Answer text preprocessing
df_train["question_preprocessed"] = df_train["question"].map(lambda x: text_preprocess(x))
df_train["answer_preprocessed"] = df_train["multiple_choice_answer"].map(lambda x: text_preprocess(x))

In [114]:
df_train

,image_id,question,multiple_choices,question_id,question_type,multiple_choice_answer,answers,answer_type,url,file_name,width,height,question_preprocessed,answer_preprocessed
0,11779,Who looks happier?,"[alive, 1, woman, purple, 2, yes, white, boy, ...",117792,who,man,"[{'answer': 'old person', 'answer_confidence':...",other,http://visualqa.org/data/abstract_v002/scene_i...,abstract_v002_train2015_000000011779.png,700,400,who looks happier,man
1,11779,Where is the woman sitting?,"[3, no, blue, red, 1, slide, monkey bars, jump...",117790,where is the,blanket,"[{'answer': 'on blanket', 'answer_confidence':...",other,http://visualqa.org/data/abstract_v002/scene_i...,abstract_v002_train2015_000000011779.png,700,400,where is the woman sitting,blanket
2,11779,Where is the man sitting?,"[away, yes, blue, 1, 2, mouse, couch, no, yell...",117791,where is the,bench,"[{'answer': 'on bench', 'answer_confidence': '...",other,http://visualqa.org/data/abstract_v002/scene_i...,abstract_v002_train2015_000000011779.png,700,400,where is the man sitting,bench
3,5536,Is this man hungry?,"[water, yellow, 4, running, blue, pouring, out...",55360,is this,yes,"[{'answer': 'yes', 'answer_confidence': 'yes',...",yes/no,http://visualqa.org/data/abstract_v002/scene_i...,abstract_v002_train2015_000000005536.png,700,400,is this man hungry,yes
4,5536,What kind of drink is that?,"[wine, girl would fall, soda, white, yes, coke...",55361,what kind of,soda,"[{'answer': 'water', 'answer_confidence': 'no'...",other,http://visualqa.org/data/abstract_v002/scene_i...,abstract_v002_train2015_000000005536.png,700,400,what kind of drink is that,soda
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59995,11695,What color is the log the little girl sitting on?,"[tan, brown, white, 1, running, red, 4, pink, ...",116950,what color is the,brown,"[{'answer': 'brown', 'answer_confidence': 'yes...",other,http://visualqa.org/data/abstract_v002/scene_i...,abstract_v002_train2015_000000011695.png,700,400,what color is the log the little girl sitting on,brown
59996,11695,Does the boy want to go on the seesaw?,"[red, no table, fetch, kicking soccer ball, ye...",116951,does the,yes,"[{'answer': 'yes', 'answer_confidence': 'yes',...",yes/no,http://visualqa.org/data/abstract_v002/scene_i...,abstract_v002_train2015_000000011695.png,700,400,does the boy want to go on the seesaw,yes
59997,13790,What animal is between the two men?,"[dog, 4, yellow, 1, collectors, no, red, yes, ...",137900,what animal is,cat,"[{'answer': 'cat', 'answer_confidence': 'yes',...",other,http://visualqa.org/data/abstract_v002/scene_i...,abstract_v002_train2015_000000013790.png,700,400,what animal is between the two men,cat
59998,13790,What food is by the picnic basket?,"[4, she is happy, grapes, pizza, white, becaus...",137901,what,watermelon,"[{'answer': 'watermelon, sandwich, burger', 'a...",other,http://visualqa.org/data/abstract_v002/scene_i...,abstract_v002_train2015_000000013790.png,700,400,what food is by the picnic basket,watermelon


In [ ]:
#create class label
one_hot_encoded_data = pd.get_dummies(data, columns = ['Remarks', 'Gender'])
print(one_hot_encoded_data)

# Preprocess the image


In [121]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import numpy as np
import nltk
import numpy as np
nltk.download('punkt')
import numpy as np
from tensorflow.keras.preprocessing import image


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [268]:
preprocess = transforms.Compose([
    transforms.Resize((28, 28)),  # Resize to match input size of the model (if needed)
    transforms.ToTensor(),  # Convert image to PyTorch tensor and normalize values between 0 and 1
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Standardize using ImageNet statistics
])

def preprocess_image(image_path):
  image = Image.open(image_path).convert("RGB")  # Convert to RGB (in case it's grayscale or has transparency)
    # Preprocess the image
  image_tensor = preprocess(image)

  # Add batch dimension (if you are feeding the image into a model)
  image_tensor = image_tensor.unsqueeze(0)
  #image_tensor = img_to_array(image_tensor)
  return image_tensor


#defin the image path
extract_dir_test = os.path.join(data_path, 'train2015/train2015_images/test')

def image_encoder(extract_dir_test):
   for filename in os.listdir(extract_dir_test):
     if filename.endswith('.png'):
       image_path = os.path.join(extract_dir_test, filename)
       tensor_gpu=preprocess_image(image)
        #tensor_result.appen[tensor_gpu]
       return tensor_gpu



In [307]:
images = []
count_id = []
for filename in os.listdir(extract_dir_test):
     # Check if the file is a PNG image
     if filename.endswith('.png'):
       # Create the full path to the image
        image_path = os.path.join(extract_dir_test, filename)
              # Open the image
        image = Image.open(image_path)
              # preprocessing image
        tensor_gpu=preprocess_image(image_path)
        images.append(tensor_gpu)
        count_id.append(filename.split('.')[0])




In [309]:
images = np.array(images)
images = images.astype('float32')
#images /= 255.0
images = tf.convert_to_tensor(images)

In [310]:
images

<tf.Tensor: shape=(18, 1, 3, 28, 28), dtype=float32, numpy=
array([[[[[ 1.42692   ,  1.3755459 ,  1.4954191 , ...,  1.4782944 ,
            1.4954191 ,  1.4954191 ],
          [ 1.3412963 ,  1.7351656 ,  2.060536  , ...,  1.0673003 ,
            1.1700488 ,  1.3241715 ],
          [ 1.2385478 ,  1.8550389 ,  2.129035  , ...,  0.31381115,
            0.2966864 ,  0.60493195],
          ...,
          [-0.9876702 , -0.9876702 , -0.9876702 , ..., -0.9876702 ,
           -0.9876702 , -0.9876702 ],
          [-0.9876702 , -0.9876702 , -0.9876702 , ..., -0.9876702 ,
           -0.9876702 , -0.9876702 ],
          [-0.9876702 , -0.9876702 , -0.9876702 , ..., -0.9876702 ,
           -0.9876702 , -0.9876702 ]],

         [[ 2.2535014 ,  1.9733893 ,  1.7983193 , ...,  2.3235295 ,
            2.3410363 ,  2.3410363 ],
          [ 2.1834733 ,  1.8333333 ,  1.7457983 , ...,  1.6582633 ,
            1.8858544 ,  2.1134453 ],
          [ 2.0784314 ,  1.8508403 ,  1.7457983 , ..., -0.495098  ,
       

In [263]:
# import pandas as pd
# import torch
# from torchvision import transforms
# from PIL import Image
# import os



# # Step 3: Initialize lists to store filenames and tensors
# filenames = []
# tensor_list = []

# # Step 4: Loop through files in the directory
# for filename in os.listdir(extract_dir_test):
#     if filename.endswith('.png') or filename.endswith('.jpg'):  # Change as needed
#         file_path = os.path.join(extract_dir_test, filename)
#         #image_path = os.path.join(extract_dir_test, filename)
#         # Load and transform the image
#         image = Image.open(file_path).convert('RGB')  # Ensure the image is in RGB format

#         image_tensor = preprocess_image(file_path).unsqueeze(0)  # Add batch dimension (1, 3, 224, 224)
#         #image_tensor = image_tensor.to(torch.uint8)


#         # Append filename and tensor to the lists
#         filenames.append(filename)
#         tensor_list.append(image_tensor)

# # Step 5: Create a DataFrame
# data = {
#     'filename': filenames,
#     'tensor': tensor_list
# }
# df = pd.DataFrame(data)

# # Step 6: Check the DataFrame
# print(df.head())
# print(df['tensor'].iloc[0].size())  # Check size of the first tensor



                                           filename  \
0  Copy of abstract_v002_train2015_000000000014.png   
1  Copy of abstract_v002_train2015_000000000000.png   
2  Copy of abstract_v002_train2015_000000000003.png   
3  Copy of abstract_v002_train2015_000000000011.png   
4  Copy of abstract_v002_train2015_000000000005.png   

                                              tensor  
0  [[[tensor([[ 1.4269,  1.3755,  1.4954,  1.3242...  
1  [[[tensor([[ 1.4954,  1.4954,  1.4954,  1.4954...  
2  [[[tensor([[ 2.2318,  2.2147,  2.1804,  2.1633...  
3  [[[tensor([[ 1.4954,  1.4954,  1.4954,  1.4954...  
4  [[[tensor([[ 1.4954,  1.4954,  1.4954,  1.4954...  
torch.Size([1, 1, 3, 28, 28])


In [285]:

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
import spacy

# Load the pre-trained spacy model for tokenization
nlp = spacy.load('en_core_web_sm')
drive.mount('/content/drive')
data_path = '/content/drive/My Drive/Colab Notebooks/'
extract_dir_test = os.path.join(data_path, 'train2015/train2015_images/test')


# Define the VQA Model
class VQAModel(nn.Module):
    def __init__(self, vocab_size, hidden_size, output_size):
        super(VQAModel, self).__init__()
        # Load pre-trained ResNet for image encoding
        self.resnet = models.resnet18(pretrained=True)
        self.resnet.fc = nn.Identity()  # Remove the final classification layer

        # LSTM for question encoding
        self.embedding = nn.Embedding(vocab_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size, batch_first=True)

        # Fully connected layers to combine image and question encodings
        self.fc1 = nn.Linear(hidden_size + 512, 512)
        self.fc2 = nn.Linear(512, output_size)

    def forward(self, image, question):
        # Image encoding
        img_features = self.resnet(image)

        # Question encoding
        question_embed = self.embedding(question)
        _, (hidden, _) = self.lstm(question_embed)

        # Combine image and question encodings
        combined = torch.cat((img_features, hidden.squeeze(0)), dim=1)

        # Pass through fully connected layers
        x = torch.relu(self.fc1(combined))
        x = self.fc2(x)

        return x


# Model Hyperparameters
hidden_size = 256
output_size = 3  # Let's say we are predicting one of 3 possible answers

# Initialize model, loss function, and optimizer
model = VQAModel(vocab_size, hidden_size, output_size)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Example Image and Question
image_path = extract_dir_test  # Replace with your image path
question = 'What is this?'
print(image_path)

# Process the inputs
image_tensor = process_image(image_path)
question_tensor = process_question(question, vocab)

# Forward pass
output = model(image_tensor, question_tensor)

# Example target (the correct answer is index 1)
target = torch.tensor([1])

# Calculate loss
loss = criterion(output, target)
print(f'Loss: {loss.item()}')

# Backpropagation and optimization
optimizer.zero_grad()
loss.backward()
optimizer.step()

print("Training step completed.")



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


/content/drive/My Drive/Colab Notebooks/train2015/train2015_images/test


FileNotFoundError: [Errno 2] No such file or directory: '/content/Copy of abstract_v002_train2015_000000000014.png'

In [303]:
 from collections import counter

ImportError: cannot import name 'counter' from 'collections' (/usr/lib/python3.10/collections/__init__.py)

step 3 question encoder


In [299]:
# Step 1: Tokenization
def tokenize_question(question):
    return [token.text.lower() for token in nlp(question)]

# Step 2: Build Vocabulary
def build_vocab(questions, vocab_threshold=2):
    counter = Counter()
    for question in questions:
        tokens = tokenize_question(question)
        counter.update(tokens)

    vocab = {word: idx+2 for idx, (word, count) in enumerate(counter.items()) if count >= vocab_threshold}
    vocab['<unk>'] = 0  # Unknown token
    vocab['<pad>'] = 1  # Padding token
    return vocab


In [300]:
# Step 3: Encode Question
def encode_question(question, vocab, max_length=10):
    tokens = tokenize_question(question)
    token_ids = [vocab.get(token, vocab['<unk>']) for token in tokens]

    # Padding
    if len(token_ids) < max_length:
        token_ids += [vocab['<pad>']] * (max_length - len(token_ids))
    else:
        token_ids = token_ids[:max_length]

    return torch.tensor(token_ids)



In [301]:
# Build Vocabulary
vocab = build_vocab(df_train["question_preprocessed"])
print("Vocabulary:", vocab)

# Encode questions
encoded_questions = [encode_question(q, vocab) for q in questions]

NameError: name 'Counter' is not defined

build answer

In [305]:
# Step 1: Build Answer Vocabulary
def build_answer_vocab(answers, vocab_threshold=1):
    counter = Counter(answers)
    vocab = {word: idx for idx, (word, count) in enumerate(counter.items()) if count >= vocab_threshold}
    return vocab

# Step 2: Encode Answer
def encode_answer(answer, vocab):
    return torch.tensor(vocab.get(answer, vocab['<unk>']))

# Build Answer Vocabulary
answer_vocab = build_answer_vocab(answers)
answer_vocab['<unk>'] = len(answer_vocab)  # Add <unk> token for unknown answers
print("Answer Vocabulary:", answer_vocab)

# Encode answers
encoded_answers = [encode_answer(a, answer_vocab) for a in answers]
print("Encoded Answers:", encoded_answers)

NameError: name 'answers' is not defined

In [ ]:
# Preprocess a question and an answer
question = "What color is the dog?"
answer = "black"

# Encode the question and answer
encoded_question = encode_question(question, vocab)
encoded_answer = encode_answer(answer, answer_vocab)

print("Encoded Question:", encoded_question)
print("Encoded Answer:", encoded_answer)